In [16]:
import requests
import pickle
import time
import json

### Set up notebook

Setting the proper parameters to get the best-fitting results from google maps

In [28]:
# the google maps api key of the app
secret_api_key = ''

# list that contains types of searched places
types = [
    'art_gallery', 'atm', 'bar', 'beauty_salon', 'book_store', 'bus_station', 
    'cafe', 'car_wash', 'casino', 'cemetery', 'church', 'city_hall', 
    'clothing_store', 'florist', 'gas_station', 'gym', 'jewelry_store', 'zoo',
    'university', 'travel_agency', 'store', 'spa', 'restaurant', 'post_office',
    'police', 'park', 'night_club', 'museum', 'library',
]  

# information about the place
city_name = 'krakow'
ltn = 50.0615868
lng = 19.9373393
radius = 5000

creation_time = int(time.time())

### Get places around a specific point

The method executes the request to google maps place API and returns the following dict: id -> (name, types, lat, lng)

In [21]:
def execute_place_request(parameters):
    response = requests.get('https://maps.googleapis.com/maps/api/place/nearbysearch/json', params=parameters)
    places = response.json()['results'] 
    result = {}
    for place in places:
        place_id = place["place_id"]
        if place_id in result:
            continue
            
        name = place["name"]
        types = place["types"]
        lat = place['geometry']['location']["lat"]
        lng = place['geometry']['location']["lng"]
        result[place_id] = (name, types, lat, lng)
    
    return result

The method executes requests for all kinds of places and returns it as a list of [id, name, types, lat, lng]

In [22]:
def load_raw_data(parameters, types):
    result = {}
    for t in types:
        parameters['type'] = t
        response = execute_place_request(parameters)
        for key, value in response.items():
            if key not in result:
                result[key] = value
                
    return [[key, value[0], value[1], value[2], value[3]] for key, value in result.items()]

Use created methods to load the data

In [23]:
parameters = {
    'location': str(ltn) + ',' + str(lng),
    'radius': radius,
    'language': 'en',
    'key': secret_api_key
}
raw_data = load_raw_data(parameters, types)

In [26]:
print(f'Downloaded information for {len(data)} places')

Downloaded information for 462 places


Save the raw data

In [30]:
with open(f'../data/places_{city_name}_raw_{creation_time}.pickle', 'wb') as file:
    pickle.dump(raw_data, file, protocol=pickle.HIGHEST_PROTOCOL)

### Download details for each loaded place

The method that sends request for place details. It returns the following list: [address, web page, google page, summary, reviews]

In [34]:
def execute_details_request(parameters):
    response = requests.get('https://maps.googleapis.com/maps/api/place/details/json', params=parameters)
    response_data = response.json()['result']
    
    place_address = None
    place_website = None
    place_url = None
    place_summary = None
    place_reviews = []

    if 'editorial_summary' in response_data:
        place_summary = response_data['editorial_summary']['overview']
    if 'reviews' in response_data:
        for review in response_data['reviews']:
            place_reviews.append(review['text'])
    if 'website' in response_data:
        place_website = response_data['website']
    if 'formatted_address' in response_data:
        place_address = response_data['formatted_address']
    if 'url' in response_data:
        place_url = response_data['url']
    
    return [place_address, place_website, place_url, place_summary, place_reviews]

The method adds details to loaded data and returns the following list: [id, name, types, lat, lng, address, web page, google page, summary, reviews]

In [35]:
def load_detailed_data(raw_data, parameters):
    result = []
    for place in raw_data:
        parameters['place_id'] = place[0]
        response = execute_details_request(parameters)
        place.extend(response)
        result.append(place)
        
    return result

Use created methods to load data details

In [36]:
parameters = {
    'key': secret_api_key,
    'language': 'en',
    'fields': 'editorial_summary,reviews,website,formatted_address,url'
}
detailed_data = load_detailed_data(raw_data, parameters)

Save the final data

In [37]:
with open(f'../data/places_{city_name}_detailed_{creation_time}.pickle', 'wb') as file:
    pickle.dump(raw_data, file, protocol=pickle.HIGHEST_PROTOCOL)